In [ ]:
import re
import json
from typing import List, Tuple
from collections import Counter
from urllib.parse import urlparse
from GlotScript import sp
from tqdm import tqdm 
import time
import string


class Filters:
    def __init__(self):
        self.filters = []

    def add_filter(self, filter_func, warning):
        self.filters.append((filter_func, warning))

    def apply_filters(self, sentence, quality_warnings):
        for filter_func, warning in self.filters:
            if filter_func(sentence):
                quality_warnings.append(warning)
        return quality_warnings

In [ ]:
CURSED_SUBSTRINGS = [" №", "���", "\\|\\s*$", " nr\\.$", "aute irure dolor ", " sunt in culpa qui ", "orem ipsum ", " quis nostrud ", " adipisicing ", " dolore eu ", " cupidatat ", "autem vel eum", "wisi enim ad", " sex ", " porn ", "黄色电影", "mp3", "ownload", "Vol\\.", " Ep\\.", "Episode", " г\\.\\s*$", " кг\\.\\s*$", " шт\\.", "Develop", "Facebook", " crusher ", " xxx ", " ... ... ... ... ... ... ... ... ...", " .... .... .... .... .... .... .... .... ....", " [^ ] [^ ] [^ ] [^ ] [^ ] [^ ] [^ ] [^ ] [^ ]", ", ..,,? ..,,? ..,,? ..,,?"]
ADULT_SIGNALS = "caoporn caoprom caopron caoporen caoponrn caoponav caopom caoorn 99re dy888 caopro hezyo re99 4438x zooskool xfplay 7tav xxoo xoxo 52av freexx 91chinese anquye cao97 538porm 87fuli 91pron 91porn 26uuu 4438x 182tv kk4444 777me ae86 91av 720lu yy6080 6080yy qqchub paa97 aiai777 yy4480 videossexo 91free 一级特黄大片 偷拍久久国产视频 日本毛片免费视频观看 久久免费热在线精品 高清毛片在线看 日本毛片高清免费视频 一级黄色录像影片 亚洲男人天堂 久久精品视频在线看 自拍区偷拍亚洲视频 亚洲人成视频在线播放 色姑娘综合站 丁香五月啪啪 在线视频成人社区 亚洲人成视频在线播放 久久国产自偷拍 一本道 大香蕉无码 香港经典三级 亚洲成在人线免费视频 天天色综合网 大香蕉伊人久草 欧美一级高清片 天天鲁夜夜啪视频在线 免费黄片视频在线观看 加比勒久久综合 久草热久草在线视频 韩国三级片大全在线观看 青青草在线视频 美国一级毛片 久草在线福利资源 啪啪啪视频在线观看免费 成人福利视频在线观看 婷婷我去也 老司机在线国产 久久成人视频 手机看片福利永久国产 高清国产偷拍在线 大香蕉在线影院 日本高清免费一本视频 男人的天堂东京热 影音先锋男人资源 五月婷婷开心中文字幕 亚洲香蕉视频在线播放 天天啪久久爱视频精品 超碰久久人人摸人人搞".split()
EXTRA_ADULT_SGINALS = "日本一级特黄大片 qq的天天中彩票 一本道 一级特黄大片 三级片 下三烂 个老子的 中国福利彩票天天 久久免费热在线精品 久久国产视频 久久精品国产 久久综合久久 乳交 乳波臀浪 五分彩 亚洲男人天堂 亚洲精品 人人摸 人人摸人人 人人操 人人澡 人人爽人人 人人碰 人人碰人人 人人碰免费公开视频 人人碰免费视频 仆街 他奶娘的 他妈 他妈ㄉ王八蛋 他妈地 他妈的 他马的 伦理在线黄影片 伦理在线黄电影 伦理在线黄网影片 伦理在线黄网电影 伦理在线黄网视频 伦理在线黄视频 伦理影片 伦理影片观看 伦理片 伦理片免费 伦理片观看 伦理电影 伦理电影片观看 伦理电影观看 伦理电影观看平台 伦理电影观看网 伦理电影观看网址 伦理电视 伦理视频 伦理视频观看 伦理黄影片 伦理黄影片观看 伦理黄片 伦理黄片网站 伦理黄电影 伦理黄电影片 伦理黄电影观看 伦理黄电视 伦理黄电视片 伦理黄网影片 伦理黄网片 伦理黄网电影 伦理黄网视频 伦理黄网视频观看 伦理黄视频 伦理黄视频片 伦理黄视频网 伦理黄视频观看 伦理黄视频频道 伦理黄频道 你个傻比 你他马的 你全家 你奶奶的 你她马的 你妈的 你娘卡好 你娘咧 你它妈的 你它马的 你是鸡 你是鸭 你马的 做爱 傻比 傻逼 免费人成视频 免费伦理 免费伦理影片 免费伦理视频 免费在线伦理黄影片 免费在线伦理黄电影 免费在线伦理黄网影片 免费在线伦理黄网电影 免费在线伦理黄网视频 免费在线伦理黄视频 免费在线成人视频 免费在线成人黄影片 免费在线成人黄电影 免费在线成人黄网影片 免费在线成人黄网电影 免费在线成人黄网视频 免费在线成人黄视频 免费在线黄影片 免费在线黄电影 免费在线黄网影片 免费在线黄网电影 免费在线黄网视频 免费在线黄视频 免费成人电影 免费成人视频 免费成人视频观看 免费无码 免费色情影片 免费色情电影 免费色情视频 免费视频在线观看 六合彩 册那 军妓 分分彩 北京赛车开奖 午夜电影 午夜福利 卖B 卖比 卖淫 博彩 口交 口肯 吃屎 吹箫 啪啪啪视频 国产av 国产精品 国产自拍 国产自拍片 国产自拍视频 在线av 在线av在线 在线av直播 在线av网址 在线av视频 在线伦理黄影片 在线伦理黄片 在线伦理黄电影 在线伦理黄网影片 在线伦理黄网电影 在线伦理黄网视频 在线伦理黄视频 在线国产 在线大香蕉 在线成人黄影片 在线成人黄片 在线成人黄电影 在线成人黄网影片 在线成人黄网电影 在线成人黄网视频 在线成人黄视频 在线观看中文字幕 在线观看免费 在线黄影片 在线黄片 在线黄电影 在线黄网影片 在线黄网电影 在线黄网视频 在线黄视频 塞你公 塞你娘 塞你母 塞你爸 塞你老师 塞你老母 夜夜啪视频在线观看 大卵子 大卵泡 大发云 大发官网 大发彩票 大发彩票官网 大发快三 大发快三和值 大发快三大小单双 大发快三如何 大发快三官网 大发快三开奖 大发快三开奖结果 大发快三怎么 大发快三怎么看 大发快三是不是 大发快三是什么 大发快三是国家 大发快三计划 大发快三走势图 大发扑克 大发时时彩 大发时时彩开奖 大发时时彩是 大发时时彩计划 大发棋牌 大发游戏 大发游戏官网 大香蕉 大香蕉伊人 大鸡巴 天天中彩票 天天中彩票app 天天中彩票微信 天天中彩票怎么 天天中彩票是 天天中彩票的 天天啪 天天啪在线视频 天天彩票 天天彩票网 天天爱彩票 天天赢彩票 夫妻性生活 奸你 她妈地 她妈的 她马的 妈B 妈个B 妈个比 妈个老比 妈妈的 妈比 妈的 妈的B 妈逼 妓 妓女 妳她妈的 妳妈的 妳娘的 妳老母的 妳马的 娱乐平台开户 娱乐彩票 开奖结果 彩神争霸 彩神争霸大发快三 彩神争霸怎么 彩神争霸是 彩神争霸网站 彩神争霸邀请码 彩票 彩票天天 彩票娱乐 彩票娱乐注册 彩票平台 彩经彩票 微信的天天中彩票 性交 性感写真 性感写真视频 性感美女 性感视频 性爱 性爱视频 情色图片 情色电影 情色网站 情色视频 情色论坛 成人下载 成人书刊 成人动漫 成人动漫网 成人图片 成人在线影片 成人在线电影 成人在线网 成人在线视频 成人在线黄影片 成人在线黄电影 成人在线黄网影片 成人在线黄网电影 成人在线黄网视频 成人在线黄视频 成人小说 成人影城 成人影片 成人影片观看 成人影视 成人文学 成人文学网 成人漫画 成人片 成人电影 成人电影下载 成人电影免费 成人电影在线 成人电影播放 成人电影片 成人电影片观看 成人电影观看 成人电影观看免费 成人电影观看平台 成人电影观看网 成人电影观看网址 成人电视 成人秀 成人网站 成人自拍 成人视频 成人视频免费网站 成人视频啪啪啪 成人视频在线 成人视频在线观看 成人视频平台 成人视频播放 成人视频片 成人视频秀 成人视频网 成人视频自拍 成人视频观看 成人视频观看免费 成人视频观看平台 成人视频观看网站 成人视频频道 成人论坛 成人频道 成人黄影片 成人黄影片观看 成人黄片 成人黄片网站 成人黄电影 成人黄电影片 成人黄电影观看 成人黄电视 成人黄电视片 成人黄网影片 成人黄网片 成人黄网电影 成人黄网视频 成人黄网视频观看 成人黄视频 成人黄视频片 成人黄视频网 成人黄视频观看 成人黄视频频道 成人黄频道 无码 无码av 无码一区二区三区 无码不卡高清免费 无码不卡高清免费v 无码中文字幕 无码国产自拍 日本一本道 日本毛片免费视频观看 日韩av 时时彩 时时彩计划 淫秽 激情小说 激情裸聊 激情裸舞 激情视频 热久久精品 热在线精品 热这里只有精品 男人的天堂 真人性生活直播 真人性生活视频 真人性直播 真人性行为直播 真人性行为视频 真人性视频 真人秀 真人秀视频 真人色情 真人色情视频 真人裸聊 真人裸舞 真人视频 真人视频秀 神彩争霸 福利彩票 福利视频 精品一区二区三区 精品国产 美女裸聊 美女裸舞 自拍视频 色情 色情图片 色情影片 色情电影 色情电影观看 色情电视 色情网 色情网站 色情视频 色情视频在线 色情视频在线观看 色情视频网 色情视频频道 色情频道 色电影 色网 色网站 色视频 色视频在线 色视频网站 赌 赌债 赌博 赌城 赌局 赌徒 赌桌 赌注 赌王 赌瘾 赌盘 赌赛 赌钱 赌鬼 重庆时时彩 重庆时时彩杀 露点 高清无码 黄影片 黄片 黄片网站 黄电影 黄电影片 黄电视 黄电视片 黄网 黄网影片 黄网片 黄网电影 黄网视频 黄色 黄色录像 黄色录像影片 黄色录像片 黄色录像片影片 黄色录像片电影 黄色录像片网影片 黄色录像片网电影 黄色录像片网视频 黄色录像片视频 黄色录像电影 黄色录像网影片 黄色录像网电影 黄色录像网视频 黄色录像视频 黄视频 黄视频片 黄视频网".split()




POLICY_SUBSTRINGS = [
    "terms of use",
    "privacy policy",
    "cookie policy",
    "uses cookies",
    "use of cookies",
    "use cookies",
]


def list_case_filter(sentence):
    tokens = sentence.split()
    capital_tokens = [token for token in tokens if token[0].isupper() or all(char.isdigit() or char in string.punctuation for char in token)]
    warning = len(tokens) >= 12 and (len(capital_tokens) / len(tokens)) > 0.5
    return warning


def danger_chars_filter(sentence):
    danger_chars_count = sum(1 for char in sentence if char in '0123456789{}+/()>')
    warning = (danger_chars_count / len(sentence)) > 0.2
    return warning


def cursedness_filter(sentence):
    warning = any(curse in sentence for curse in CURSED_SUBSTRINGS)
    return warning

def adult_filter(sentence):
    warning = any(curse in sentence for curse in ADULT_SIGNALS + EXTRA_ADULT_SGINALS)
    return warning

def detect_long_words(sentence, max_chars=100):
    words = sentence.split()
    long_words = [word for word in words if len(word) > max_chars]
    return len(long_words) > 0

def detect_js_warning(sentence):

    # Check if "Javascript" is present in the text
    if 'Javascript' in sentence or 'JavaScript' in sentence:
        return True
    
    return False


def detect_lorem_ipsum(sentence):
    # Convert text to lowercase to make the search case-insensitive
    sentence = sentence.lower()
    
    # Check if the phrase "lorem ipsum" is present in the text
    if "lorem ipsum" in sentence:
        return True
    
    return False


def detect_curly_bracket(sentence):
    # Check if the curly bracket '{' is present in the text
    if "{" in sentence:
        return True
    
    return False


def detect_policy(text):
    text_lower = text.lower()
    for substring in POLICY_SUBSTRINGS:
        if substring in text_lower:
            return True
    return False


In [ ]:
class GopherRepetitionFilter():
    
    """Check if there is repeated content in the input text. Excessive
    repetition is often linked with uninformative content and can be used to
    determine whether it is low-quality text. This function implements
    "Repetition Removal" as described in Gopher_.

    .. _Gopher: https://arxiv.org/abs/2112.11446
    """
    
    @staticmethod
    def get_n_grams(words: List[str], n: int) -> List[str]:
        return [" ".join(words[i : i + n]) for i in range(len(words) - n + 1)]

    @staticmethod
    def find_top_duplicate(x: List[str]) -> int:
        counter = Counter()
        for element in x:
            counter[element] += 1
        top_n_gram = counter.most_common(1)[0]
        return len(top_n_gram[0]) * top_n_gram[1]


    def filter(self, text, top_n_grams):
        
        text = self.add_space_between_numbers(text)
        words = text.split(' ')

        for n, n_frac in top_n_grams:
            n_grams = self.get_n_grams(words, n)
            if not n_grams:
                continue
            top_char_length = self.find_top_duplicate(n_grams)
            if top_char_length / len(text) > n_frac:
                return True

        return False

    
    @staticmethod
    def add_space_between_numbers(numbers):
        result = ''
        i = 0
        while i < len(numbers):
            if numbers[i].isdigit():
                result += numbers[i]
                i += 1
                while i < len(numbers) and numbers[i].isdigit():
                    result += numbers[i]
                    i += 1
                if i < len(numbers) and not numbers[i].isdigit():
                    result += ' '
            else:
                result += numbers[i]
                if i+1 < len(numbers) and numbers[i+1].isdigit():
                    result += ' '
                i += 1
        return result
    
    def filter_para(self, text):
        
        return self.filter(text, ((2, 0.2), (3, 0.18), (4, 0.16)))
    
    def filter_sent(self, text):
        
        sents = [s for s in text.split('\n') if len(s.split(' ')) > 20]
        warning = any([self.filter(s, ((1, 0.5), (2, 0.3))) for s in sents])
        
        return warning

gopher_repetition = GopherRepetitionFilter().filter_para
gopher_repetition_sent = GopherRepetitionFilter().filter_sent

In [ ]:
filters = Filters()
filters.add_filter(list_case_filter, "list_case")
filters.add_filter(danger_chars_filter, "danger_chars")
filters.add_filter(cursedness_filter, "cursed_regex")
filters.add_filter(adult_filter, "adult_signals")
filters.add_filter(detect_long_words, "long_word")
filters.add_filter(detect_policy, "detect_policy")
filters.add_filter(gopher_repetition, "repetition")
filters.add_filter(gopher_repetition_sent, "repetition_sent")
filters.add_filter(detect_js_warning, "js_warning")
filters.add_filter(detect_lorem_ipsum, "lorem_ipsum")
filters.add_filter(detect_curly_bracket, "curly_bracket")

In [ ]:
class DomainLabeler:
    def __init__(self):
        self.domain_groups = {}
    
    def add_domain_group(self, label, domains):
        self.domain_groups[label] = domains
    
    def apply(self, uri, categories):
        domain = urlparse(uri).netloc
        for label, domains in self.domain_groups.items():
            if any(domain.endswith(d) for d in domains):
                categories.append(label)
                break
        return categories


labeler = DomainLabeler()
labeler.add_domain_group("religious", ["bible.com", "ebible.org", "png.bible", "jw.org", "wol.jw.org", "breakeveryyoke.com", "scriptureearth.org", "live.bible.is", "bible.is", "faithcomesbyhearing.com", "download.sabda.org", "sabda.org", "alkitab.mobi", "biblerevelation.org", "gospelgo.com", "mykitabsuci.org", "aboriginalbibles.org.au", "wikiislam.net", "stepbible.org", "e-alkitab.org"])
labeler.add_domain_group("wikipedia", ["wikipedia.org", "wikimedia.org"])

In [ ]:
import ipaddress
import re
from functools import partial


class PIIReplacer:
    def __init__(
        self, regex: str, replacements, validator
    ):
        self.regex: re.Pattern = re.compile(regex)
        self.replacements = (
            replacements
            if type(replacements) is tuple
            else (tuple(replacements) if not isinstance(replacements, str) else (replacements,))
        )
        self.validator = validator  # extra validation for a match
        self._replace_i = 0

    def replace(self, text: str):
        def get_replacement(matchobj):
            if self.validator and not self.validator(matchobj.group(0)):
                # not a valid match. replace with itself
                return matchobj.group(0)
            replacement = self.replacements[self._replace_i]
            self._replace_i = (self._replace_i + 1) % len(self.replacements)
            return replacement

        return self.regex.sub(get_replacement, text)


def public_ip_validator(ip, public_only: bool = True) -> bool:
    try:
        ip = ipaddress.ip_address(ip)
        return not public_only or ip.is_global
    except ValueError:
        return False


class PIIFormatter():
    """
    Replaces email addresses and ip addresses in the document text.
    Args:
        remove_emails: Replace email addresses
        remove_ips: Replace IP addresses
        only_remove_public_ips: by default we only replace public (and thus PII) IPs
        email_replacement: tuple of strings to use as replacement. They will be used in a circular way
        ip_replacement same as email_replacement but for IP addresses
    """

    name = "📞 PII"

    def __init__(
        self,
        remove_emails: bool = True,
        remove_ips: bool = True,
        only_remove_public_ips: bool = True,
        # example.com/org are actually maintained as an example
        email_replacement  = ("email@example.com", "firstname.lastname@example.org"),
        # randomly generated list of ips. they did not respond to ping requests at the time the list was created
        ip_replacement = (
            "22.214.171.124",
            "126.96.36.199",
            "188.8.131.52",
            "184.108.40.206",
            "220.127.116.11",
            "18.104.22.168",
        ),
    ):
        super().__init__()
        self.remove_emails = remove_emails
        self.remove_ips = remove_ips

        self.emails_replacer = PIIReplacer(
            r"\b[A-Za-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[A-Za-z0-9!#$%&'*+/=?^_`{|}~-]+)*@(?:(?:[A-Za-z0-9](?:["
            r"A-Za-z0-9-]*[A-Za-z0-9])?\.)+[A-Za-z0-9](?:[A-Za-z0-9-]*[A-Za-z0-9])?|\[(?:(?:25[0-5]|2[0-4][0-9]|["
            r"01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[A-Za-z0-9-]*[A-Za-z0-9]:)])",
            email_replacement,
            None
        )

        self.ip_replacer = PIIReplacer(
            r"(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)",
            validator=partial(public_ip_validator, public_only=only_remove_public_ips),
            replacements=ip_replacement,
        )

    def format(self, text: str) -> str:
        if self.remove_emails:
            text = self.emails_replacer.replace(text)
        if self.remove_ips:
            text = self.ip_replacer.replace(text)
        return text
    
    
pii_format = PIIFormatter().format

In [ ]:
def detect_script(text, lang):
    
    main_script, percentage, details = sp(text)
    if lang == 'jpn':
        main_script = 'Jpan'
        percentage = details['details'].get('Hani', 0) + details['details'].get('Hira', 0) + details['details'].get('Kana', 0)
        
    return main_script, percentage

In [ ]:
def process_json(json_data, lang, script):
    
    # pii
    json_data["content"] = pii_format(json_data["content"])
    content = json_data["content"]
    
    warc_headers = json_data["warc_headers"]
    uri = warc_headers["warc-target-uri"]

    metadata = json_data["metadata"]
    
    quality_warnings = metadata.get("quality_warnings", [])
    categories = metadata.get("categories", [])
    
    if categories is None:
        categories = []

    if quality_warnings is None:
        quality_warnings = []
        
    quality_warnings = filters.apply_filters(content, quality_warnings)
    categories = labeler.apply(uri, categories)

    metadata["quality_warnings"] = sorted(set(quality_warnings))
    metadata["categories"] = sorted(set(categories))

    
    # run script identification
    script_label, script_percentage = detect_script(content, lang)
    metadata["script"] = {"label": script_label, "percentage": float("{:.2f}".format(script_percentage))}

    
    ## lang identification consistency
    
    label = metadata['identification']['label']
    sent_labels = [int(isent['label']==label) for isent in metadata['sentence_identifications'] if isinstance(isent, dict)]
    metadata['identification_consistency'] = {"percentage": float("{:.2f}".format(sum(sent_labels)/len(sent_labels))), 'num_sents': len(sent_labels)}
    
    
    return json_data



In [ ]:
def Hani_nonspace_percentage(text):
    # Split the text into words
    
    words = text.split()
    
    # Count the number of words with 10 or more characters
    long_words_length = sum(len(word) for word in words if len(word) >= 20)
    
    # Calculate the percentage
    percentage = long_words_length / len(text)
    
    return percentage

In [ ]:
def post_annotation_filter(json_data, lang, script):
    
    metadata = json_data['metadata']
    
    if 'wikipedia' in metadata["categories"]:
        quality = set(metadata['quality_warnings']) - set(["short_sentences","header", "footer", "tiny", "long_word", "repetition", "repetition_sent"])
    
    # tiny for oscar is based on 5 sentences, we decrease it to 3
    if 'tiny' in metadata['quality_warnings']:
        
        if metadata['identification_consistency']['num_sents'] >= 3:
            metadata['quality_warnings'].remove('tiny')
    
    
    if metadata['script']['label'] != script and script not in ['Hani', 'Hans', 'Hant']:
        return False
    
    if script in ['Hani', 'Hans', 'Hant'] and metadata['script']['label'] not in ['Hani', 'Hans', 'Hant']:
        return False
    
    if metadata['script']['percentage'] < 0.9:
        return False
    
    if lang != 'und' and metadata['identification_consistency']['percentage'] < 0.6:
        return False

    if script in ['Hani', 'Jpan']:
        
        if Hani_nonspace_percentage(json_data['content']) < 0.3:
            metadata['quality_warnings'].append('hani_list_case')
          
        
        # for chinese and japanse do not remove tiny. 
        quality = set(metadata['quality_warnings']) - set(["short_sentences","header", "footer", "tiny", "long_word", "repetition", "repetition_sent"])


    if script in ['Latn', 'Cyrl', 'Arab', 'Grek', 'Hebr', 'Deva', 'Beng']:
        quality = set(metadata['quality_warnings']) - set(["short_sentences","header", "footer"])

    else:
        quality = set(metadata['quality_warnings']) - set(["short_sentences","header", "footer", "long_word", "repetition", "repetition_sent"])
    
    
    # quality = quality - set(['curly_bracket'])
    
    if len(quality)!=0:
        return False
    
    
    return True

In [ ]:
import json
import os
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

# Batch size
batch_size = 2000


# Function to process JSON data
def process_json_file(file_path):
    output_file_path = os.path.join(output_dir, os.path.basename(file_path))
    filter_file_path = os.path.join(filter_dir, os.path.basename(file_path))

    lang =  os.path.basename(file_path).split('-')[0]
    script = os.path.basename(file_path).split('-')[-1].split('_')[0]
    
    if lang == 'multi':
        return None
    
    with open(file_path, 'r', encoding='utf-8') as input_file, open(output_file_path, 'w', encoding='utf-8') as output_file, open(filter_file_path, 'w', encoding='utf-8') as filter_file:
        batch = []
        batch_filter = []
        for line in tqdm(input_file, desc=f'Processing {file_path}'):
            # Parse the line as JSON
            try:
                data = json.loads(line)
            except:
                continue 
            # Process the JSON object
            processed_data = process_json(data, lang, script)
            # Add processed data to the batch
            batch.append(processed_data)

            if post_annotation_filter(processed_data, lang, script):
                batch_filter.append(processed_data)

            # If the batch is full, write it to the output file
            if len(batch) == batch_size:
                for item in batch:
                    output_file.write(json.dumps(item, ensure_ascii=False) + '\n')
                batch = []

            # If the batch_filter is full, write it to the output file
            if len(batch_filter) == batch_size:
                for item in batch_filter:
                    filter_file.write(json.dumps(item, ensure_ascii=False) + '\n')
                batch_filter = []

        # Write remaining data in the last batch to the output file
        for item in batch:
            output_file.write(json.dumps(item, ensure_ascii=False) + '\n')

        # Write remaining data in the last filter_batch to the output file
        for item in batch_filter:
            filter_file.write(json.dumps(item, ensure_ascii=False) + '\n')


In [ ]:
num_cores = 40

In [ ]:
import os

# Function to filter paths based on file size
def filter_paths_by_size(paths, max_size_mb):
    max_size_bytes = max_size_mb * 1024 * 1024  # Convert MB to bytes
    filtered_paths = [path for path in paths if os.path.isfile(path) and os.path.getsize(path) < max_size_bytes]
    return filtered_paths

input_dir = 'res/corpus/'
output_dir = 'res/annotation/'
filter_dir = 'res/filter/'

# Get list of input files
input_files = [os.path.join(input_dir, file) for file in os.listdir(input_dir) if file.endswith('.jsonl')]
input_files = sorted(input_files, key=os.path.getsize)

input_files_100mgb = filter_paths_by_size(input_files, 100)

rest_input_files = list(set(input_files) - set(input_files_100mgb))

In [ ]:
# Define the number of processes to run in parallel
num_processes = min(len(input_files_100mgb), num_cores)

# Create a pool of processes
with Pool(processes=num_processes) as pool:
    # Map the processing function to each input file
    pool.map(process_json_file, input_files_100mgb)


In [ ]:
import multiprocessing as mp
import subprocess

def process_batch(lines, lang, script):
    
    batch = []
    batch_filter = []
    for line in lines:
        # Parse the line as JSON
        try:
            data = json.loads(line)
        except:
            continue 
        # Process the JSON object
        processed_data = process_json(data, lang, script)
        # Add processed data to the batch
        batch.append(processed_data)

        if post_annotation_filter(processed_data, lang, script):
            batch_filter.append(processed_data)
            
    return batch, batch_filter


def check_and_remove_empty_file(file_path):
    """
    Check if a file at file_path is empty and remove it if it is.
    
    Parameters:
    file_path (str): The path to the file to be checked and potentially removed.
    """
    if os.path.isfile(file_path) and os.path.getsize(file_path) == 0:
        os.remove(file_path)


def process_chunk(start_line, end_line, input_file_path, chunk_index, batch_size=2000):
    
    print("chunk_index", chunk_index)
    
    output_file_path = os.path.join(output_dir, os.path.basename(input_file_path).replace('.jsonl', f'_{chunk_index}.jsonl'))
    filter_file_path = os.path.join(filter_dir, os.path.basename(input_file_path).replace('.jsonl', f'_{chunk_index}.jsonl'))
    
    with open(input_file_path, 'r', encoding='utf-8') as infile, open(output_file_path, 'w', encoding='utf-8') as outfile, open(filter_file_path, 'w', encoding='utf-8') as filterfile:
        
        lang =  os.path.basename(input_file_path).split('-')[0]
        script = os.path.basename(input_file_path).split('-')[-1].split('_')[0]

        
        current_line = 0
        lines = []
        
        for line in infile:
            if current_line >= start_line:
                lines.append(line)
            current_line += 1

            if current_line >= end_line:
                break

            if len(lines) == batch_size:
                # Process the lines (replace this with your actual processing function)
                processed_lines, processed_filtered_lines  = process_batch(lines, lang, script)
                                
                for item in processed_lines:
                    outfile.write(json.dumps(item, ensure_ascii=False) + '\n')

                for item in processed_filtered_lines:
                    filterfile.write(json.dumps(item, ensure_ascii=False) + '\n')

                lines = []

        # Process any remaining lines in the last batch
        if lines:
            processed_lines, processed_filtered_lines  = process_batch(lines, lang, script)

            for item in processed_lines:
                outfile.write(json.dumps(item, ensure_ascii=False) + '\n')

            for item in processed_filtered_lines:
                filterfile.write(json.dumps(item, ensure_ascii=False) + '\n')


    check_and_remove_empty_file(output_file_path)
    check_and_remove_empty_file(filter_file_path)
    
    return True
        
        
        
# def count_lines(file_path):
#     result = subprocess.run(['sed', '-n', '$=', file_path], capture_output=True, text=True)
#     return int(result.stdout.strip())
              

def count_lines(filename):
    count = 0
    with open(filename, 'rb') as f:
        while chunk := f.read(1024*1024*1024):
            count += chunk.count(b'\n')
    return count        

def chunkify(file_path, num_chunks):
    # with open(file_path, 'r') as infile:
    #    total_lines = sum(1 for _ in infile)
    print(file_path)
    total_lines = count_lines(file_path)
    chunk_size = total_lines // num_chunks
    chunks = []
    for i in range(num_chunks):
        start_line = i * chunk_size
        end_line = start_line + chunk_size if i != num_chunks - 1 else total_lines
        chunks.append((start_line, end_line))
    
    return chunks

def parallel(input_file, num_cores):
    
    print("start", input_file)
    chunks = chunkify(input_file, num_cores)
    print("chunks", len(chunks))
    
    processes = []
    for chunk_index, (start_line, end_line) in enumerate(chunks):
        p = mp.Process(target=process_chunk, args=(start_line, end_line, input_file, chunk_index))
        processes.append(p)
        p.start()
    
    for p in processes:
        p.join()


In [ ]:
rest_input_files = sorted(rest_input_files, key=os.path.getsize)

for r in rest_input_files:
    parallel(r, num_cores)
